In [1]:
import requests
import datetime
import bs4
import time
import sqlite3
from sqlite3 import Error

In [2]:
stationData=[{ 'value': '115', 'text': 'Bendung. Cibalok - Gadog' }, { 'value': '116', 'text': 'Bendung. Katulampa (Hulu)' }, { 'value': '118', 'text': 'PS. Depok' }, { 'value': '122', 'text': 'PA. Manggarai' }, { 'value': '114', 'text': 'PS. Krukut Hulu' }, { 'value': '144', 'text': 'Pompa Cideng' }, { 'value': '143', 'text': 'P.A. Karet' }, { 'value': '140', 'text': 'P.A. Marina Ancol ' }, { 'value': '117', 'text': 'Pompa Pasar Ikan' }, { 'value': '141', 'text': 'Pompa. Pluit' }, { 'value': '108', 'text': 'PS. Pesanggrahan' }, { 'value': '106', 'text': 'PS. Angke Hulu' }, { 'value': '135', 'text': 'PS. Sunter Hulu' }, { 'value': '126', 'text': 'PA. Pulo Gadung' }, { 'value': '142', 'text': 'Pompa Yos Sudarso 1' }, { 'value': '103', 'text': 'PS. Cipinang Hulu' }, { 'value': '145', 'text': 'Pompa Kali Duri (Kalijodo)' }, { 'value': '148', 'text': 'P.A. Istiqlal  ' }, { 'value': '147', 'text': 'P.A. Jembatan Merah' }, { 'value': '146', 'text': 'P.A. Flusing Ancol' }, { 'value': '149', 'text': 'P.A. Hek' }]

In [6]:
def dbDsda():
    return r"dsda.db"


def create_connection(db_file):
    '''
    create a database connection to a SQLite database
    specified by db_file
    :param db_file : database file
    :return: Connection Object or None
    '''
    conn=None
    try:
        conn=sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
        
def create_table(conn, create_table_sql):
    '''
    create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    '''
    try:
        c=conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_create_dsdadb(conn, stationName):
    sql_create_table="""CREATE TABLE IF NOT EXISTS {}(currentdate text UNIQUE, data text)""".format(stationName)
    
    #create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_table)
        print("Success")
    else:
        print("Error! cannot create the database conenction.")
        
def insert_dsdadb(conn, table, datevalues, datavalues):
    '''
    Insert new datevalues to dsda table
    :param conn:
    :param datevalues: text in sqlite datetime format
    '''
    sql='''insert into {}(currentdate, data) values("{}","{}")'''.format(table,datevalues,datavalues)
    cur=conn.cursor()
    try:
        cur.execute(sql,)
        conn.commit()
        return "Data insert success"
    except Error:
        return "Data insert Error - Already inserted"
        

def currentStartEndDate(stationCode, yearList, totalData, currentLoop):
    '''
    Return a pair of tuple data containing start and end date for data crawling.
    Also, by passing stationCode (unique), this function will store current loop performed on the disk
    '''
    
    
    #print(currentLoop)

    # from 2000 to 2021
    #yearList=['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']

    # month : 01 - 12
    monthList=['01','02','03','04','05','06','07','08','09','10','11','12']

    # daylist : 
    startDay='01'
    endDayListKabisat=['31','29','31','30','31','30','31','31','30','31','30','31']
    endDayListN=['31','28','31','30','31','30','31','31','30','31','30','31']


    #currentLoop=0
    #totalData=len(yearList)*len(monthList)
    monthIndex=currentLoop%12
    yearIndex=currentLoop//12
    if monthIndex == 12:
        monthIndex=0
        yearIndex=yearIndex+1   


    if currentLoop < totalData:
        '''
        Performing loop in current Year list to a 1-month-basis for crawling purpose
        '''
        # START DATE
        # assign data section
        year=yearList[yearIndex]
        month=monthList[monthIndex]

        # get day data based on month and 'kabisat' year identification
        checkRemainder=int(year)%4
        if checkRemainder != 0:
            day=endDayListN[monthIndex]
        else:
            day=endDayListKabisat[monthIndex]

        # increment section
        if monthIndex < 11:
            monthIndex+=1
        else:
            monthIndex=0
            yearIndex+=1
        currentLoop+=1
        
        # first tuple index is for identification, 0 if the loop is ongoing, 1 if we are already on the end of the loop
        return ('0','{}'.format(currentLoop),'{}/{}/{}'.format(startDay,month,year), '{}/{}/{}'.format(day,month,year))
        
    else: # else we already on the end of the loop
        return ('1','0','0','0') 
        
def dsdaRequestData(currentStation, yearList, totalData, currentLoop):
    '''
    Return dictionary of data from dsda website for current loop and station
    '''
    check, currentLoopStatus, startDate, endDate = currentStartEndDate(currentStation, yearList, totalData, currentLoop)
    print('{}/{} : {}-{}'.format(currentLoop+1, totalData, startDate, endDate))
    if check=='0':
        url='http://poskobanjirdsda.jakarta.go.id/Pages/GenerateDataTinggiAir.aspx?IdPintuAir={}&StartDate={}&EndDate={}'.format(currentStation,startDate,endDate)
        # test URL down below (worked!)
        #url='http://poskobanjirdsda.jakarta.go.id/Pages/GenerateDataTinggiAir.aspx?IdPintuAir=122&StartDate=02/03/2021&EndDate=03/03/2021'
        response=requests.get(url)
        
        # start processing response content
        # beautify text got from the response content
        soup = bs4.BeautifulSoup(response.content)
        textOnly=soup.text
        # split with "|"
        twoKindData=textOnly.split("|")
        mainData=twoKindData[0]
        # split again with ';'
        mainDataLists=mainData.split(';')
        # delete last item in list, because contained empty data
        mainDataLists.pop(len(mainDataLists)-1)
        dataDictionary={}
        for data in mainDataLists:
            # last split with ','
            dateText, dataText=data.split(',')
            dateFormatted='{}:{}'.format(dateText[:13], dateText[14:16])
            dataDictionary[dateFormatted]=dataText
        
    else:
        # update current loop on file with 0 again (meaning that we are ready start again from zero)
        with open('dcl-{}.cache'.format(stationCode),'w') as dcl:
            dcl.write('0')
        
    if response.ok:
        # update current loop on file
        with open('dcl-{}.cache'.format(currentStation),'w') as dcl:
            dcl.write(str(currentLoopStatus))
            
        return dataDictionary
    
    elif not response.ok:
        # update current loop on file with last failed fetch batch
        with open('dcl-{}.cache'.format(currentStation),'w') as dcl:
            dcl.write(str(currentLoopStatus-1))
        
        return 0
    

    
def executeBatchPastDsdaData(currentStation, stationName, yearList):
    '''
    yearList=['2000','2001','2002','2003','2004','2005',
    '2006','2007','2008','2009','2010','2011','2012','2013',
    '2014','2015','2016','2017','2018','2019','2020','2021']
    
    '''
    # check stored current loop
    try:
        with open('dcl-{}.cache'.format(currentStation),'r') as dcl:
            fetchCurrentLoop=dcl.read()
            currentLoop=int(fetchCurrentLoop)
    except:
        currentLoop=0
        with open('dcl-{}.cache'.format(currentStation),'w') as dcl:
            dcl.write(str(currentLoop))
    
    totalData=len(yearList)*12
    
    while currentLoop < totalData:
        print('Start fetching data')
        start_time=time.time()
        conn=create_connection(dbDsda())        
        datas=dsdaRequestData(currentStation, yearList, totalData, currentLoop)
        
        # if error occured, continue on the same batch of data after sleep
        if datas == 0:
            print("Failed to reach server. Trying again in 30 seconds")
            time.sleep(30)
            continue
        
        # refetching current station progress
        try:
            with open('dcl-{}.cache'.format(currentStation),'r') as dcl:
                fetchCurrentLoop=dcl.read()
                currentLoop=int(fetchCurrentLoop)
        except:
            currentLoop=0
            with open('dcl-{}.cache'.format(currentStation),'w') as dcl:
                dcl.write(str(currentLoop))
        
        
        
        execute_create_dsdadb(conn, stationName)
        for data in datas:
            insertData=insert_dsdadb(conn, stationName, data, datas[data])
            
        # performing sleep for every fetch loop to prevent abusive behaviour
        finish_time=time.time()
        print('Finish data processing in {} secs, sleep for 10s'.format(finish_time-start_time))
        time.sleep(4)
        
    
        
    

In [7]:
crawledStation={'122':'manggarai'}
yearList=['2013','2014','2015','2016','2017','2018','2019','2020','2021']
for station in crawledStation:
    print('Crawling {} started'.format(crawledStation[station]))
    executeBatchPastDsdaData(station, crawledStation[station], yearList)
    print('Finish crawling {}!'.format(crawledStation[station]))

Crawling manggarai started
Start fetching data
97/108 : 01/01/2021-31/01/2021
Success
Finish data processing in 0.981351375579834 secs, sleep for 10s
Start fetching data
98/108 : 01/02/2021-28/02/2021
Success
Finish data processing in 1.2838597297668457 secs, sleep for 10s
Start fetching data
99/108 : 01/03/2021-31/03/2021
Success
Finish data processing in 3.410123348236084 secs, sleep for 10s
Start fetching data
100/108 : 01/04/2021-30/04/2021
Success
Finish data processing in 0.21550512313842773 secs, sleep for 10s
Start fetching data
101/108 : 01/05/2021-31/05/2021
Success
Finish data processing in 3.0860893726348877 secs, sleep for 10s
Start fetching data
102/108 : 01/06/2021-30/06/2021
Success
Finish data processing in 0.16283440589904785 secs, sleep for 10s
Start fetching data
103/108 : 01/07/2021-31/07/2021
Success
Finish data processing in 0.1652839183807373 secs, sleep for 10s
Start fetching data
104/108 : 01/08/2021-31/08/2021
Success
Finish data processing in 0.1437733173370